In [1]:
print("oui")

oui


In [1]:
import tensorflow as tf
import os
import PIL
import numpy as np

In [2]:
IMAGE_SIZE = (2792,99)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (99,2792,1) # shape qui rentre dans le cnn
# même taille pour toutes les parties d'images, présentes en même densité

THRESHOLD_PREDICTION = 0.5

NUM_CALLS = tf.data.experimental.AUTOTUNE

DATA_PATH = "/home/cogrannr/roues/MEFRO/grises/"
DATA_PATH_DEFAUTS_REELS = "/home/cogrannr/roues/MEFRO/images_defauts/"
DATA_PATH_ROUES_ENTIERES="/home/cogrannr/roues/MEFRO/Images_Roues/Actuelle/gris_blanc/"

# Preprocessing

In [3]:
"""
    Fonction custom, car doit être fait en eager execution, sinon c'est un param "none" passé à Image.open
    ouvereture des photos / resize
"""
def open_image(file_name): 
    name = tf.get_static_value(file_name)
    name = name.decode()
    
    image = PIL.Image.open(name)
    
    image = image.resize(IMAGE_SIZE)
    image_arr = np.array(image)
    
    image_arr = image_arr[:,:,np.newaxis]
    return image_arr

def open_image_entiere(file_name): 
    name = tf.get_static_value(file_name)
    name = name.decode()
    
    image = PIL.Image.open(name)
    
    image = image.resize(IMAGE_SIZE)
    image_arr = np.array(image)
    return image_arr
    
def parse_images(filename,label): # appel "open_image"
    image_arr = tf.py_function(open_image,[filename],tf.float32)
    image_arr = tf.image.convert_image_dtype(image_arr,tf.float32)
    return image_arr,label

def parse_images_entieres(filename,label): # appel "open_image"
    image_arr = tf.py_function(open_image_entiere,[filename],tf.float32)
    image_arr = tf.image.convert_image_dtype(image_arr,tf.float32)
    return image_arr,label

def data_augmentation(image,label):
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_brightness(image, max_delta=10.0/255.0)
    #image = tf.clip_by_value(image, 0.0, 1.0)

    return image,label
    
def reset_shapes_gray(image,label):
    image.set_shape(list(INPUT_IMAGE_SHAPE))
    return image, label


# Generate datasets

In [4]:
"""
    chargement des listes de fichiers photo
"""
def load_data(cover_dir,faulty_dir): 
    
    file_names_cover = tf.data.Dataset.list_files(cover_dir+"/*",shuffle=False)
    file_names_cover = file_names_cover.take(DATA_SIZE//2)
    labels_0 = tf.data.Dataset.from_tensors(0).repeat()
    data_set_cover = tf.data.Dataset.zip((file_names_cover,labels_0))
    
    file_names_faulty = tf.data.Dataset.list_files(faulty_dir+"/*",shuffle=False)
    file_names_faulty = file_names_faulty.take(DATA_SIZE//2)
    labels_1 = tf.data.Dataset.from_tensors(1).repeat()
    data_set_faulty = tf.data.Dataset.zip((file_names_faulty,labels_1))
   
    data_set = data_set_cover.concatenate(data_set_faulty)
    
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
   
    return data_set

def load_data_entieres(cover_dir,faulty_dir, num_negative, num_positive): 
    
    file_names_cover = tf.data.Dataset.list_files(cover_dir,shuffle=False)
    labels_0 = tf.data.Dataset.from_tensors(0).repeat()
    
    data_set_cover = tf.data.Dataset.zip((file_names_cover,labels_0))
    data_set_cover = data_set_cover.take(num_negative)
    
    
    file_names_faulty = tf.data.Dataset.list_files(faulty_dir,shuffle=False)
    labels_1 = tf.data.Dataset.from_tensors(1).repeat()
    
    data_set_faulty = tf.data.Dataset.zip((file_names_faulty,labels_1))
    data_set_faulty = data_set_faulty.take(num_positive)
    
    data_set = data_set_cover.concatenate(data_set_faulty)
    
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
   
    return data_set


""" 
    répartition des fichiers enn ensembles train/valid
"""
def generate_train_valid_test(cover_dir,faulty_dir, augment_data=False, entieres=False,
                             num_positive=0,num_negative=0): 
    
    if entieres==True:
        data_set = load_data_entieres(cover_dir, faulty_dir, num_negative,num_positive)
    else:
        data_set = load_data(cover_dir, faulty_dir)
        
    train_dataset = data_set.take(DATA_TRAIN_SIZE)
    valid_dataset = data_set.skip(DATA_TRAIN_SIZE).take(DATA_VALID_SIZE)
    test_dataset = data_set.skip(DATA_TRAIN_SIZE+DATA_VALID_SIZE).take(DATA_TEST_SIZE)

    
    datasets_list = []
    
    for data_set in [train_dataset, valid_dataset, test_dataset]:

        if data_set == train_dataset:
            if entieres == True:
                data_set = data_set.map(parse_images_entieres, num_parallel_calls=NUM_CALLS)
            else:
                data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
            if augment_data == True:
                data_set = data_set.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)
                data_set = data_set.map(data_augmentation, num_parallel_calls=NUM_CALLS)
        else:
            if entieres == True:
                data_set = data_set.map(parse_images_entieres, num_parallel_calls=NUM_CALLS)
            else:
                data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
            
        data_set = data_set.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)
            
        data_set = data_set.batch(BATCH_SIZE)
        data_set = data_set.prefetch(1)
        
        datasets_list.append(data_set)
        
    return datasets_list

In [5]:
# confusion matrix : 

def confusion_matrix(test_dataset,model, num_positive_class, num_negative_class):
    
    prediction = np.array([])
    labels = tf.constant([], dtype=tf.int32)

    for (data_batch, label_batch) in test_dataset:
        prediction = np.concatenate((prediction, model.predict_on_batch(data_batch)[:,0]))
        labels = tf.concat((labels, label_batch), axis=0)
  
    prediction[prediction > THRESHOLD_PREDICTION] = 1
    prediction[prediction <= THRESHOLD_PREDICTION] = 0
        
    confusion_matrix = tf.math.confusion_matrix(
        labels=labels,predictions=tf.convert_to_tensor(prediction),num_classes=2)

    l = tf.get_static_value(labels)
    
    num_pos = np.sum(l==1)
    num_neg = np.sum(l==0)
    print("num pos ",num_pos)
    print("num neg ",num_neg)
    
    nomralized_matrix = tf.get_static_value(confusion_matrix)
    nomralized_matrix = nomralized_matrix.astype(np.float32)
    
    nomralized_matrix[0,:] = nomralized_matrix[0,:]/num_neg
    nomralized_matrix[1,:] = nomralized_matrix[1,:]/num_pos

    print("\nConfusion matrix : ")
    print(tf.get_static_value(confusion_matrix))
    print("\nNormalized confusion matrix : ")
    print(nomralized_matrix)

 

# Model architecture

In [6]:
def init_model(architecture=1):
    
    if architecture == 1:    # 1M param : MARCHE OK
        model_input = tf.keras.Input(shape=INPUT_IMAGE_SHAPE)
        conv_1 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,2))(model_input)
        conv_2 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_1)
        bn_1 = tf.keras.layers.BatchNormalization()(conv_2)
        mxp_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_1)
        
        conv_3 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,1))(mxp_1)
        conv_4 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_3)
        bn_2 = tf.keras.layers.BatchNormalization()(conv_4)
        mxp_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_2)
        
        conv_5 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,1))(mxp_2)
        conv_6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_5)
        bn_3 = tf.keras.layers.BatchNormalization()(conv_6)
        mxp_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_3)
        
        conv_7 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='SAME',strides=(1,1))(mxp_3)
        conv_8 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_7)
        bn_4 = tf.keras.layers.BatchNormalization()(conv_8)
        mxp_4 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_4)
        
        flat = tf.keras.layers.Flatten()(mxp_4)
        drop = tf.keras.layers.Dropout(0.3)(flat)
        out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)
    

    model = tf.keras.Model(inputs=model_input, outputs=out)
    print(model.summary())
    
    return model

# Training settings 

In [7]:
def train_model(model,train_dataset,valid_dataset, check_point_name, num_epochs,tensor_board_name=""):
    
    precision_metric = tf.keras.metrics.Precision(thresholds=THRESHOLD_PREDICTION)
    
    call_backs_list = [
        #tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience = 2),
        tf.keras.callbacks.ModelCheckpoint(
            filepath =check_point_name, monitor ="val_accuracy", save_best_only = True),
    ]
    
    if tensor_board_name != "":
        board = tf.keras.callbacks.TensorBoard(log_dir='./tensor_board_'+tensor_board_name)
        call_backs_list.append(board)
   
    optim = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
  
    model.compile(loss='binary_crossentropy',optimizer=optim,
        metrics=[precision_metric,"accuracy"]) 
    
    model.fit(train_dataset,epochs=num_epochs,
              validation_data=valid_dataset,
             callbacks=call_backs_list
             )
    
    return model

# Train Images gray


In [7]:
DATA_SIZE = int(146e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)


LEARNING_RATE = 0.0001 

NUM_EPOCHS = {"galbe":10, "ajours":10, "crochet":10, "central":150}

SAVE_PATH = "./"

NUM_POSITIVE_CLASS = DATA_TEST_SIZE//2
NUM_NEGATIVE_CLASS = DATA_TEST_SIZE//2

BATCH_SIZE = 16

MODEL_NAME = "base_model_"

In [15]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():

    for wheel_part in ["galbe"]:# "ajours", "crochet", "central"]:
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts")

        model = init_model(architecture=1)
        model = train_model(model, train_dataset, valid_dataset,
                        check_point_name=SAVE_PATH+MODEL_NAME+wheel_part+".h5", num_epochs=NUM_EPOCHS[wheel_part])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 99, 2792, 1)]     0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 99, 1396, 32)      320       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 99, 1396, 32)      9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 99, 1396, 32)      128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 49, 698, 32)       0         
____________________________________________________________

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5475/5475 [==============================] - 434s 79ms/step - loss: 0.0785 - precision_2: 0.9883 - accuracy: 0.9795 - val_loss: 0.1046 - val_precision_2: 0.9909 - val_accuracy: 0.9674
Epoch 10/10
2251/5475 [===========>..................] - ETA: 3:35 - loss: 0.0713 - precision_2: 0.9895 - accuracy: 0.9801

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
# pour la RAM : bien shutdown le notebook avant cette partie !

for wheel_part in ["galbe"]:#, "ajours", "crochet", "central"]:
    
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts")
    
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    
    print("evaluating resnet model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    #confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)
    

evaluating resnet model for wheel part : galbe
1825/1825 [==============================] - 62s 34ms/step - loss: 0.0579 - precision_2: 0.9937 - accuracy: 0.9836


In [9]:
import numpy as np
import anchors


INPUT_IMAGE_SHAPE = (99,2790,1)

RATIO_X = 4
RATIO_Y = 8

anchor_sizes = ((10,20),
            (20,40),
            (35,70),
            (50,100),
            (10,10),
            (15,15),
            (20,20),
            (25,25),
            (30,30),
            (40,40),
            (50,50))


bbox_list = np.genfromtxt("bbox_galbe_2.csv",delimiter=",",dtype=np.int32)


anchors_arr = anchors.generate(INPUT_IMAGE_SHAPE, RATIO_X, RATIO_Y, anchor_sizes)

#positive_anchors_index, negative_anchors_index = compute_IoU(anchors,bbox_list[0],positive_threshold=0.6)

#positive_anchors_index, negative_anchors_index = select_anchors(10,10,positive_anchors_index,negative_anchors_index)
print(anchors_arr.shape)
#draw_anchors("photo.pgm",np.reshape(a,(-1,4)))
anchors.debug_compute_number_positive_anchors(anchor_sizes,anchors_arr,"bbox_galbe_2.csv")



(8352, 11, 4)
% of positives anchors >= 5, for every truth bbox : [0.86396928]


In [8]:
for wheel_part in ["galbe"]:# "ajours", "crochet", "central"]:
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts")

    for (x,y) in train_dataset:
        break
    print(x)

tf.Tensor(
[[[[ 88.]
   [ 85.]
   [ 86.]
   ...
   [ 88.]
   [ 88.]
   [ 88.]]

  [[ 94.]
   [ 92.]
   [ 91.]
   ...
   [ 91.]
   [ 93.]
   [ 94.]]

  [[101.]
   [ 99.]
   [ 97.]
   ...
   [ 96.]
   [ 99.]
   [101.]]

  ...

  [[194.]
   [196.]
   [198.]
   ...
   [196.]
   [194.]
   [194.]]

  [[194.]
   [197.]
   [200.]
   ...
   [195.]
   [193.]
   [194.]]

  [[194.]
   [197.]
   [200.]
   ...
   [194.]
   [193.]
   [194.]]]


 [[[173.]
   [174.]
   [176.]
   ...
   [177.]
   [175.]
   [173.]]

  [[179.]
   [181.]
   [185.]
   ...
   [188.]
   [182.]
   [178.]]

  [[192.]
   [195.]
   [199.]
   ...
   [199.]
   [195.]
   [192.]]

  ...

  [[ 92.]
   [ 95.]
   [100.]
   ...
   [ 92.]
   [ 92.]
   [ 92.]]

  [[138.]
   [138.]
   [138.]
   ...
   [137.]
   [137.]
   [137.]]

  [[157.]
   [157.]
   [157.]
   ...
   [158.]
   [157.]
   [157.]]]


 [[[214.]
   [212.]
   [209.]
   ...
   [205.]
   [211.]
   [215.]]

  [[213.]
   [213.]
   [213.]
   ...
   [214.]
   [214.]
   [213.]]

  [[2